In [108]:
import os 
import sys
import datetime
import csv 

import cirq
import qsimcirq
import openfermion
import numpy as np
import multiprocessing as mp
Pi=3.1415


In [109]:
from functools import partial

## anzats

In [110]:
class AnzatsAFMHeisenberg():
    def __init__(self, length, gamma, beta):

        # initialize circuit
        circuit = cirq.Circuit()
        qubits = cirq.LineQubit.range(length)


        for i in range(int(length/2)):
            circuit.append(cirq.H(qubits[int(2*i)]))
            circuit.append(cirq.Y(qubits[int(2*i)]))
            circuit.append(cirq.X(qubits[int(2*i+1)]))
            circuit.append(cirq.CNOT(qubits[int(2*i)], qubits[int(2*i+1)]))

        # add gamma circuit
        for index in range(gamma.size):
            # add gates on 2n and 2n+1
            for i in range(1, length-1, 2):
                circuit.append(
                    cirq.XX(qubits[i], qubits[(i+1)]) ** (-gamma[index]*2/Pi)
                    )
                circuit.append(
                    cirq.YY(qubits[i], qubits[(i+1)]) ** (-gamma[index]*2/Pi)
                    )
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)]) ** (-gamma[index]*2/Pi)
                    )
            # add gates on 2n-1 and 2n

            # add beta circuit
            for i in range(0, length-1, 2):
                circuit.append(
                    cirq.XX(qubits[i], qubits[(i+1)]) ** (-beta[index]*2/Pi)
                    )
                circuit.append(
                    cirq.YY(qubits[i], qubits[(i+1)]) ** (-beta[index]*2/Pi)
                    )
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)]) ** (-beta[index]*2/Pi)
                    )

        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta



## expectation

In [111]:
class AFMHeisenbergArgs():
    def __init__(self, length, qsim_option):
        self.length = length
        self.qsim_option = qsim_option


def get_expectation_afm_heisenberg(function_args, gamma, beta):
    anzats = AnzatsAFMHeisenberg(function_args.length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    simulator = qsimcirq.QSimSimulator(function_args.qsim_option)
    vector = simulator.simulate(circuit).state_vector()

    value = 0 + 0j
    for i in range(function_args.length-1):
        circuitX = anzats.circuit.copy()
        circuitY = anzats.circuit.copy()
        circuitZ = anzats.circuit.copy()

        circuitX.append(cirq.X(qubits[i]))
        circuitX.append(cirq.X(qubits[(i+1)]))
        vector2 = simulator.simulate(circuitX).state_vector()
        value += np.dot(vector2.conj(), vector)

        circuitY.append(cirq.Y(qubits[i]))
        circuitY.append(cirq.Y(qubits[(i+1)]))
        vector2 = simulator.simulate(circuitY).state_vector()
        value += np.dot(vector2.conj(), vector)

        circuitZ.append(cirq.Z(qubits[i]))
        circuitZ.append(cirq.Z(qubits[(i+1)]))
        vector2 = simulator.simulate(circuitZ).state_vector()
        value += np.dot(vector2.conj(), vector)
    return value


## optimization

In [112]:
def get_gradient(function, gamma: np.array, beta: np.array, delta_gamma, delta_beta, iter):
    grad_gamma = np.zeros_like(gamma)
    grad_beta  = np.zeros_like(beta)
    gamma_edge = gamma
    beta_edge  = beta
    # initial gamma, beta?
    
    if not (gamma.size == beta.size):
        return 1

    for index in range(gamma.size):
        center = gamma[index]
        gamma_edge[index] = gamma[index] - delta_gamma
        e1 = function(gamma=gamma_edge, beta=beta)
        gamma_edge[index] = gamma[index] + delta_gamma
        e2 = function(gamma=gamma_edge, beta=beta)
        grad_gamma[index] = (e2.real-e1.real)/(2*delta_gamma)
        gamma[index] = center

        center = beta[index]
        beta_edge[index] = beta[index] - delta_beta
        e1 = function(gamma=gamma, beta=beta_edge)
        beta_edge[index] = beta[index] + delta_beta
        e2 = function(gamma=gamma, beta=beta_edge)
        grad_beta[index] = (e2.real-e1.real)/(2*delta_beta)
        beta[index] = center
    
    return grad_gamma, grad_beta


def optimize_by_gradient_descent(function, initial_gamma: np.array, initial_beta: np.array, alpha, delta_gamma, delta_beta, iteration, figure=True,filepath=""):
    gamma, beta = initial_gamma, initial_beta

    textlines = []
    headline = ["iter", "energy"]
    for p in range(int(len(initial_gamma))):
        headline.append("gamma[{}]".format(p))
        headline.append("bata[{}]".format(p))
    print(headline)
    textlines.append(headline)

    for iter in range(int(iteration)):
        # it is complex for me to set get_gradient for two optical parameter_vector
        grad_gamma, grad_beta = get_gradient(function, gamma, beta, delta_gamma, delta_beta, iter)
        gamma -= alpha * grad_gamma
        beta  -= alpha * grad_beta
        energy = function(gamma=gamma, beta=beta)

        record = [iter, energy]
        for index in range(gamma.size):
            record.append(gamma[index])
            record.append(beta[index])
        textlines.append(record)
        print(record)
    
    if len(filepath)>0:
        with open(filepath, mode='a') as f:
            writer = csv.writer(f)
            for i, textline in enumerate(textlines):
                writer.writerow(textline)
                # f.write("{}\n".format(textline))

    return gamma, beta


## case: length 2, optimize gamma and beta

In [113]:
length = 2
gamma = np.array([2.0])
beta  = np.array([2.0])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

anzats = AnzatsAFMHeisenberg(length, gamma, beta)
circuit = anzats.circuit
print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_afm_heisenberg(function_args,gamma, beta)
print("expectation on afm: {}".format(value))

initial_gamma = np.array([0.1, 0.2])
initial_beta  = np.array([0.1, 0.2])
iteration = 100
alpha = 0.1
delta_gamma = 0.0001
delta_beta  = 0.0001
gamma, beta = optimize_by_gradient_descent(partial(get_expectation_afm_heisenberg, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

0: ───H───Y───@───XX──────────YY──────────ZZ──────────
              │   │           │           │
1: ───X───────X───XX^(8/11)───YY^(8/11)───ZZ^(8/11)───
measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1))
output vector: (0.379-0.597j)|01⟩ + (-0.379+0.597j)|10⟩

phase:
output vector: |⟩
norm: (0.9999999403953552+0j)
expectation on afm: (-5.999998569488525+0j)
['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]']
[0, (-6.000002145767212+0j), 0.1, 0.09892711639404297, 0.2, 0.1994635581970215]
[1, (-5.999999642372131+0j), 0.1, 0.09928474426269532, 0.2, 0.19982118606567384]
[2, (-6.000000715255737+0j), 0.1, 0.09964237213134766, 0.2, 0.2007152557373047]
[3, (-5.999998927116394+0j), 0.1, 0.10053644180297852, 0.2, 0.20035762786865235]
[4, (-5.999999284744263+0j), 0.1, 0.10143051147460938, 0.2, 0.20107288360595704]
[5, (-5.999999284744263+0j), 0.1, 0.1, 0.2, 0.2007152557373047]
[6, (-6.000002861022949+0j), 0.1, 0.09892711639404297, 0.2, 0.2]
[7, (-5.99999

[10, (-5.999998927116394+0j), 0.1, 0.09839067459106446, 0.2, 0.19928474426269532]
[11, (-5.999998927116394+0j), 0.1, 0.0987483024597168, 0.2, 0.1987483024597168]
[12, (-6+0j), 0.1, 0.0987483024597168, 0.2, 0.19856948852539064]
[13, (-6.000000715255737+0j), 0.1, 0.09803304672241211, 0.2, 0.19785423278808595]
[14, (-5.999998927116394+0j), 0.1, 0.09982118606567383, 0.2, 0.19785423278808595]
[15, (-5.999999284744263+0j), 0.1, 0.10035762786865235, 0.2, 0.1982118606567383]
[16, (-5.999998927116394+0j), 0.1, 0.1, 0.2, 0.19785423278808595]
[17, (-6+0j), 0.1, 0.09910593032836915, 0.2, 0.1969601631164551]
[18, (-5.999999642372131+0j), 0.1, 0.0987483024597168, 0.2, 0.1962449073791504]
[19, (-5.999999642372131+0j), 0.1, 0.0987483024597168, 0.2, 0.1962449073791504]
[20, (-5.999999642372131+0j), 0.1, 0.0987483024597168, 0.2, 0.1962449073791504]
[21, (-5.999999642372131+0j), 0.1, 0.0987483024597168, 0.2, 0.1962449073791504]
[22, (-5.999999642372131+0j), 0.1, 0.0987483024597168, 0.2, 0.196244907379150

## case: length 4, static gamma and beta

In [114]:
length = 4
g = 0.8
gamma = np.array([0.5, 0.4, 0.3, 0.2])
beta  = np.array([0.7, 0.5, 0.3, 0.1])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

anzats = AnzatsAFMHeisenberg(length, gamma, beta)
circuit = anzats.circuit
print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_afm_heisenberg(function_args,gamma, beta)
print("expectation on afm: {}".format(value))

0: ───H───Y───@───────────────────────────────────────XX──────────YY──────────ZZ──────────────────────────────────────────────XX──────────YY──────────ZZ──────────────────────────────────────────────XX──────────YY──────────ZZ──────────────────────────────────────────────XX──────────YY──────────ZZ──────────
              │                                       │           │           │                                               │           │           │                                               │           │           │                                               │           │           │
1: ───X───────X───XX──────────YY──────────ZZ──────────XX^-0.446───YY^-0.446───ZZ^-0.446───XX──────────YY──────────ZZ──────────XX^-0.318───YY^-0.318───ZZ^-0.318───XX──────────YY──────────ZZ──────────XX^-0.191───YY^-0.191───ZZ^-0.191───XX──────────YY──────────ZZ──────────XX^-0.064───YY^-0.064───ZZ^-0.064───
                  │           │           │                                               

## case: length 8, static gamma and beta (paper value)

In [115]:
length = 8
gamma = np.array([0.2496, 0.4808, 0.5260, 0.4503])
beta  = np.array([0.6845, 0.6559, 0.6048, 0.3180])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

anzats = AnzatsAFMHeisenberg(length, gamma, beta)
circuit = anzats.circuit
# print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_afm_heisenberg(function_args,gamma, beta)
print("expectation on afm: {}".format(value))

measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4), cirq.LineQubit(5), cirq.LineQubit(6), cirq.LineQubit(7))
output vector: [-1.01321511e-08+8.27530577e-09j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j -5.94119864e-09-1.03792019e-09j
  0.00000000e+00+0.00000000e+00j -2.07100612e-08+1.80451032e-09j
  8.16860002e-09-6.26723606e-09j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  9.03858322e-10+9.60362634e-09j
 -2.95023672e-09+7.41995532e-09j  0.00000000e+00+0.00000000e+00j
  5.58793545e-09+1.49011612e-08j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  3.76827009e-02+2.39322037e-02j
  0.00000000e+00+0.00000000e+00j  1.52950363e-08+5.03630382e-09j
 -7.49248930e-09+7.59884333e-09j  0.00000000e+00+0.00000000e+00j
  2.32837483e-09-1.23189436e-09j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j -4.59804572e-02-8.79628435e-02j
 -1.6569451

## case: length 4, optimize beta and gamma

In [116]:
length = 4
initial_gamma = np.array([1.7, 1.7])
initial_beta  = np.array([1.7, 1.7])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)
iteration = 100
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_afm_heisenberg, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]']
[0, (-7.948523640632629+0j), 1.7178837776184082, 1.7076466798782348, 1.7047135353088378, 1.7250899791717529]
[1, (-7.984689712524414+5.960464477539063e-08j), 1.7286537408828735, 1.7027298927307128, 1.7152719020843505, 1.72566397190094]
[2, (-7.995490312576294+2.9802322387695312e-08j), 1.7347745418548584, 1.702681016921997, 1.7194215774536132, 1.7303518772125244]
[3, (-7.998804569244385+5.960464477539063e-08j), 1.73814697265625, 1.7015833973884582, 1.722541582584381, 1.7313818454742431]
[4, (-7.999698638916016-3.725290298461914e-08j), 1.739951503276825, 1.7014296174049377, 1.7240293145179748, 1.7326323509216308]
[5, (-7.999934911727905-8.195638656616211e-08j), 1.740903389453888, 1.7012049078941345, 1.7250282883644104, 1.7330972671508789]
[6, (-7.999985456466675-1.4901161193847656e-08j), 1.7413718819618225, 1.7011149048805236, 1.725598406791687, 1.7334316492080688]
[7, (-7.999980449676514-3.725290298461914e-08j), 1.74158109

KeyboardInterrupt: 

## case: length 8, optimize beta and gamma

In [ ]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

iteration = 10
alpha = 0.001
delta_gamma = 0.001
delta_beta  = 0.001

# print("exact energy: ")
# print(-1.28*length)

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_afm_heisenberg, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]', 'gamma[2]', 'bata[2]', 'gamma[3]', 'bata[3]']
[0, (-8.339006152004004-1.955777406692505e-08j), 0.4914024490863085, 0.5132264662533998, 0.5018216259777546, 0.5136496815830469, 0.5028650257736444, 0.5123851168900728, 0.49070058204233646, 0.5012611392885447]
[1, (-9.63614897057414-5.587935447692871e-08j), 0.48278083093464375, 0.5252001415938139, 0.5040623303502798, 0.5276437029242516, 0.5050302352756262, 0.5251512918621302, 0.4811694510281086, 0.5018834266811609]
[2, (-10.656131029129028+1.280568540096283e-08j), 0.47481832653284073, 0.5348623935133219, 0.5072556603699923, 0.5404608827084303, 0.5071414280682802, 0.5369598902761936, 0.4721646402031183, 0.5016213059425354]
[3, (-11.350390937179327+6.332993507385254e-08j), 0.46808947809040546, 0.5417153285816312, 0.5115873860195279, 0.5509266322478652, 0.5097008515149355, 0.5467914417386055, 0.4642690820619464, 0.5004437072202563]
[4, (-11.80169004574418+4.6566128730773926e-09j)

## case: length 8, optimize beta and gamma b

In [ ]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

iteration = 20
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

# print("exact energy: ")
# print(-1.28*length)

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_afm_heisenberg, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]', 'gamma[2]', 'bata[2]', 'gamma[3]', 'bata[3]']
[0, (-8.027773283421993+2.2351741790771484e-08j), 0.4140244908630848, 0.6322646625339985, 0.5182162597775459, 0.6364968158304691, 0.5286502577364445, 0.6238511689007282, 0.40700582042336464, 0.5126113928854465]
[1, (-7.5949688740074635+1.4901161193847656e-07j), 0.4664292298257351, 0.48830897733569145, 0.612159252166748, 0.5216475687921047, 0.5865858234465122, 0.4667520634829998, 0.44380416348576546, 0.4380754865705967]
[2, (-11.798904063180089+0j), 0.4520861543715, 0.5785348191857338, 0.5780762173235416, 0.5440720841288567, 0.5923328287899494, 0.555072158575058, 0.3956098034977913, 0.4320741184055805]
[3, (-9.864982821047306+4.6566128730773926e-09j), 0.4382064864039421, 0.574977558106184, 0.5352821592241526, 0.6572581753134727, 0.531253956258297, 0.6271305046975613, 0.3974491283297539, 0.46210379526019096]
[4, (-7.21214759349823-2.9802322387695312e-08j), 0.45012030005455017, 0

KeyboardInterrupt: 

## case: length 16, static beta and gamma

In [ ]:
length = 16
p = 8
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

iteration = 10
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

# print("exact energy: ")
# print(-1.28*length)

value = get_expectation_afm_heisenberg(function_args,gamma, beta)
print("expectation on afm: {}".format(value))

expectation on afm: (-1.1298944614827633-1.3562384992837906e-08j)


## playground

In [ ]:
L = 10
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(L)

for i in range(1, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
        )
    )


for i in range(2, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
            )
    )


for i in range(L):
    circuit.append(
        cirq.XPowGate(
            exponent=0.5, global_shift=0.0).on(
            qubits[i])
    )


# for i in range(1, L, 2):
#     circuit.append(cirq.Z(qubits[i-1]))
#     circuit.append(cirq.Z(qubits[i]))


# for i in range(2, L, 2):
#     circuit.append(cirq.Y(qubits[i-1]))
#     circuit.append(cirq.Y(qubits[i]))


# for i in range(L):
#     circuit.append(cirq.X(qubits[i]))



print(circuit)


0: ───ZZ───────X^0.5────────────
      │
1: ───ZZ^0.5───ZZ───────X^0.5───
               │
2: ───ZZ───────ZZ^0.5───X^0.5───
      │
3: ───ZZ^0.5───ZZ───────X^0.5───
               │
4: ───ZZ───────ZZ^0.5───X^0.5───
      │
5: ───ZZ^0.5───ZZ───────X^0.5───
               │
6: ───ZZ───────ZZ^0.5───X^0.5───
      │
7: ───ZZ^0.5───ZZ───────X^0.5───
               │
8: ───ZZ───────ZZ^0.5───X^0.5───
      │
9: ───ZZ^0.5───X^0.5────────────


In [ ]:
value = 0 + 0j

# circuit2 = circuit
# anzats2 = Anzats(length, gamma, beta)
# circuit2 = anzats2.circuit
# qubits = anzats2.qubits
# circuit2.append(
#         cirq.ZZPowGate(
#             exponent=1, global_shift=0.0).on(
#                 qubits[1], qubits[2]
#         ))
# print(circuit2)
# result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
# vector2 = cirq.final_state_vector(circuit2)
# value -= np.dot(vector2.conjugate(), vector)

# print(value)

# # for i in range(length-1):
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[i], qubits[i+1]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)
# else:
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[length-1], qubits[0]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)

In [ ]:
a = np.array([1,3])
a[1]

3

In [ ]:
print(vector)

[0.4591058 -4.49337065e-03j 0.12919496-1.25825405e-03j
 0.12919495-1.25825033e-03j 0.07737847-7.35387206e-04j
 0.12919495-1.25830155e-03j 0.04038734-3.95298004e-04j
 0.07737847-7.35415146e-04j 0.06066927-5.82884997e-04j
 0.12919495-1.25823729e-03j 0.03783051-3.70055437e-04j
 0.04038734-3.95270996e-04j 0.02739706-2.70117074e-04j
 0.07737848-7.35405833e-04j 0.02739704-2.70104036e-04j
 0.06066925-5.82892448e-04j 0.05631588-5.69688156e-04j
 0.12919493-1.25823729e-03j 0.03746548-3.55772674e-04j
 0.03783053-3.70066147e-04j 0.0245581 -2.27299519e-04j
 0.04038735-3.95267271e-04j 0.01372327-1.31545588e-04j
 0.02739703-2.70092394e-04j 0.02465377-2.33075581e-04j
 0.0773785 -7.35389534e-04j 0.02455811-2.27298588e-04j
 0.02739705-2.70094723e-04j 0.02209694-2.07859091e-04j
 0.06066927-5.82899433e-04j 0.02465375-2.33095139e-04j
 0.05631589-5.69676980e-04j 0.06066924-5.82894310e-04j
 0.12919495-1.25824474e-03j 0.03783055-3.70055437e-04j
 0.03746548-3.55740078e-04j 0.0245581 -2.27305107e-04j
 0.0378305

In [ ]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

p = 1
initial_gamma = np.array([-3.0 for i in range(p)])
initial_beta  = np.array([4.0 for i in range(p)])

alpha = 0.1
delta_gamma = 0.0001
delta_beta = 0.0001
iteration = 100

final_gamma, final_beta = optimize_by_gradient_descent(function_norm, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration, Figure=True)

print(final_gamma, final_beta)

['iter', 'energy', 'gamma[0]', 'bata[0]']
[0, 20.250009000048443, -2.6999950000002855, 3.6000049999995696]
[1, 16.402515390162296, -2.4299904999994766, 3.2400094999975835]
[2, 13.286044756250423, -2.186986449998926, 2.9160135499979276]
[3, 10.761702813850011, -1.9682828049986085, 2.6244171949971715]
[4, 8.71698518447511, -1.771449524498145, 2.3619804754972904]
[5, 7.060763314251251, -1.5942995720487048, 2.1257874279467757]
[6, 5.71922306798135, -1.4348646148437645, 1.9132136851514012]
[7, 4.632574990255437, -1.2913731533593626, 1.721897316635964]
[8, 3.7523896168740816, -1.1622308380235786, 1.5497125849721591]
[9, 3.039439077053899, -1.0460027542212842, 1.3947463264749569]
[10, 2.461948791156269, -0.9413974787991748, 1.2552766938276303]
[11, 1.9941813457998523, -0.8472527309192319, 1.1297540244450364]
[12, 1.6152894326596015, -0.7625224578273722, 1.0167836220005242]
[13, 1.3083867288548008, -0.6862652120446207, 0.9151102598005076]
[14, 1.0597953100277273, -0.617633690840111, 0.82360423

In [ ]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

initial_gamma = np.array([3.0, 2.0])
initial_beta = np.array([4.0, 5.0])

final_gamma, final_beta = optimize_by_gradient_descent(
    function=function_norm,
    initial_gamma=initial_gamma,
    initial_beta=initial_beta,
    alpha=0.1,
    delta_gamma=0.001,
    delta_beta=0.001,
    iteration=100,
    Figure=True)
print(final_gamma, final_beta)
print(function_norm(gamma, beta))

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]']
[0, 43.741260009996445, 2.7000499999998624, 3.6000500000000955, 1.8000499999999846, 4.500049999999618]
[1, 35.43155463610003, 2.4300949999999517, 3.2400950000003945, 1.6200949999998642, 4.0500949999997715]
[2, 28.700579899441102, 2.1871354999997124, 2.9161355000003795, 1.4581354999997558, 3.6451354999999808]
[3, 23.248388317329418, 1.9684719499996035, 2.6245719500004725, 1.3123719499998003, 3.2806719500002757]
[4, 18.83202129493629, 1.7716747549995944, 2.3621647550002365, 1.1811847549998404, 2.9526547549999904]
[5, 15.254681350009095, 1.5945572794996394, 2.1259982795000596, 1.0631162794997522, 2.657439279499947]
[6, 12.35696160350955, 1.4351515515496267, 1.9134484515502557, 0.9568546515497083, 2.391745351550174]
[7, 10.00974165684386, 1.2916863963946952, 1.7221536063953344, 0.8612191863947665, 2.1526208163951743]
[8, 8.108433243244505, 1.162567756755248, 1.5499882457558698, 0.7751472677553366, 1.9374087347556923]
[9, 6.56

In [ ]:
length = 6
gamma = np.array([0.1, 0.1])
beta = np.array([0.1, 0.1])

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
qubits = anzats.qubits
vector = cirq.final_state_vector(circuit)

ham_nd = np.zeros((length, length), dtype=np.complex128)

print(cirq.Z(qubits[-1]))
for i in range(length-1):
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[i]))
    circuit_ZiZj.append(cirq.Z(qubits[i+1]))
    ham_nd += cirq.unitary(circuit_ZiZj)
else:
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[length-1]))
    circuit_ZiZj.append(cirq.Z(qubits[0]))
    ham_nd += cirq.unitary(circuit_ZiZj)
    
print(ham_nd)


# def get_expectation_ZiZj(length, beta, gamma, qubit_ZZ_on_i, qubit_ZZ_on_j, print_circuit=False):
#     """
#     controls of append on circuit are accumulated.
#     we have to make circuits on each hamiltonian terms.
#     """
#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     vector = cirq.final_state_vector(circuit)

#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_i]))
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_j]))
#     if print_circuit:
#         print(circuit)
#     simulator = cirq.Simulator()
#     # result2 = simulator.simulate(circuit, qubit_order=qubits)
#     vector2 = cirq.final_state_vector(circuit)
#     return np.dot(vector2.conjugate(), vector)


# def get_expectation_ghz_l4(gamma: np.array, beta: np.array):
#     length = 4
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


# def get_expectation_ghz_l8(gamma: np.array, beta: np.array):
#     length = 8
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


Z(q(5))


ValueError: operands could not be broadcast together with shapes (6,6) (4,4) (6,6) 